# Treinamento com Unsloth para classificação de toxicidade (TOLD-BR)

Notebook para fine-tuning supervisionado (SFT) com LoRA/QLoRA usando o dataset `JAugusto97/told-br` do Hugging Face.

## 1) Instalação de dependências (se necessário)

> Descomente a célula abaixo se estiver em ambiente limpo.

In [ ]:
# %pip install -U unsloth datasets trl transformers accelerate bitsandbytes

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
from trl import SFTConfig, SFTTrainer
from unsloth import FastLanguageModel

## 2) Configurações

In [ ]:
config = {
    "dataset_name": "JAugusto97/told-br",
    "dataset_config": None,
    "model_name": "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "output_dir": "outputs/toldbr-unsloth",
    "max_seq_length": 1024,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "grad_accum": 4,
    "epochs": 2,
    "learning_rate": 2e-4,
    "warmup_ratio": 0.05,
    "lora_r": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.0,
    "seed": 42,
    "test_size": 0.1,
}

## 3) Funções auxiliares

In [ ]:
PROMPT_TEMPLATE = """### Instrução:
Classifique a mensagem abaixo como TOXICA ou NAO_TOXICA.

### Mensagem:
{texto}

### Resposta:
{rotulo}"""

TEXT_CANDIDATES = ("text", "texto", "tweet", "sentence", "comment", "content")
LABEL_CANDIDATES = ("label", "toxic", "toxicity", "classe", "class", "target")

def find_column(columns, candidates, kind):
    for candidate in candidates:
        if candidate in columns:
            return candidate

    lowered = {column.lower(): column for column in columns}
    for candidate in candidates:
        if candidate in lowered:
            return lowered[candidate]

    raise ValueError(
        f"Não foi possível identificar a coluna de {kind}. Colunas disponíveis: {list(columns)}"
    )

def normalize_label(value):
    if isinstance(value, bool):
        return "TOXICA" if value else "NAO_TOXICA"

    if isinstance(value, int):
        return "TOXICA" if value == 1 else "NAO_TOXICA"

    text_value = str(value).strip().lower()
    toxic_aliases = {"1", "toxic", "toxica", "tóxica", "hate", "abusive"}
    return "TOXICA" if text_value in toxic_aliases else "NAO_TOXICA"

def format_example(example, text_column, label_column):
    texto = str(example[text_column]).strip()
    rotulo = normalize_label(example[label_column])
    return {"text": PROMPT_TEMPLATE.format(texto=texto, rotulo=rotulo)}

def ensure_train_eval(dataset, test_size, seed):
    if isinstance(dataset, DatasetDict):
        if "train" in dataset and "validation" in dataset:
            return DatasetDict(train=dataset["train"], validation=dataset["validation"])
        if "train" in dataset and "test" in dataset:
            return DatasetDict(train=dataset["train"], validation=dataset["test"])
        if "train" in dataset:
            split = dataset["train"].train_test_split(test_size=test_size, seed=seed)
            return DatasetDict(train=split["train"], validation=split["test"])

    if isinstance(dataset, Dataset):
        split = dataset.train_test_split(test_size=test_size, seed=seed)
        return DatasetDict(train=split["train"], validation=split["test"])

    raise ValueError("Não foi possível construir splits de treino/validação para o dataset informado.")

## 4) Carregar e preparar o dataset

In [ ]:
raw_dataset = load_dataset(config["dataset_name"], config["dataset_config"])
dataset = ensure_train_eval(raw_dataset, test_size=config["test_size"], seed=config["seed"])

text_column = find_column(dataset["train"].column_names, TEXT_CANDIDATES, "texto")
label_column = find_column(dataset["train"].column_names, LABEL_CANDIDATES, "rótulo")

train_dataset = dataset["train"].map(
    format_example,
    fn_kwargs={"text_column": text_column, "label_column": label_column},
    remove_columns=dataset["train"].column_names,
    desc="Formatando treino",
)

eval_dataset = dataset["validation"].map(
    format_example,
    fn_kwargs={"text_column": text_column, "label_column": label_column},
    remove_columns=dataset["validation"].column_names,
    desc="Formatando validação",
)

print(train_dataset[0]["text"][:300])

## 5) Carregar modelo e aplicar LoRA

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config["model_name"],
    max_seq_length=config["max_seq_length"],
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=config["lora_r"],
    lora_alpha=config["lora_alpha"],
    lora_dropout=config["lora_dropout"],
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    use_gradient_checkpointing="unsloth",
    random_state=config["seed"],
)

## 6) Configurar trainer e treinar

In [ ]:
training_args = SFTConfig(
    output_dir=config["output_dir"],
    dataset_text_field="text",
    per_device_train_batch_size=config["train_batch_size"],
    per_device_eval_batch_size=config["eval_batch_size"],
    gradient_accumulation_steps=config["grad_accum"],
    learning_rate=config["learning_rate"],
    num_train_epochs=config["epochs"],
    warmup_ratio=config["warmup_ratio"],
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    seed=config["seed"],
    fp16=False,
    bf16=True,
    optim="adamw_8bit",
    lr_scheduler_type="cosine",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_args,
    max_seq_length=config["max_seq_length"],
    packing=False,
)

trainer.train()

## 7) Salvar artefatos

In [ ]:
trainer.save_model(config["output_dir"])
tokenizer.save_pretrained(config["output_dir"])
print(f"Treinamento concluído. Artefatos em: {config['output_dir']}")